In [ ]:
import configparser
import json
from datetime import datetime

import get_files
import redis

from pony import orm
from pony.orm import *
from pony.orm.core import EntityMeta

In [13]:
set_sql_debug(True)

In [14]:
config = configparser.ConfigParser()config.read("config.ini")

['config.ini']

In [16]:
db = Database()
provider = "mysql"


class Media(db.Entity):
    id = PrimaryKey(int, auto=True)
    xxhash = Required(str, 16, unique=True)
    size = Optional(float)
    exif = Optional(Json)


db.bind(
    provider=provider,
    host=config[provider]["host"],
    user=config[provider]["user"],
    passwd=config[provider]["pass"],
    db="media_index4",
)
db.generate_mapping(create_tables=True)


GET NEW CONNECTION
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SET foreign_key_checks = 0
CREATE TABLE `media` (
  `id` INTEGER PRIMARY KEY AUTO_INCREMENT,
  `xxhash` VARCHAR(16) UNIQUE NOT NULL,
  `size` DOUBLE,
  `exif` JSON
)

SELECT `media`.`id`, `media`.`xxhash`, `media`.`size`, `media`.`exif`
FROM `media` `media`
WHERE 0 = 1

COMMIT
SET foreign_key_checks = 1
CLOSE CONNECTION


In [18]:
config = configparser.ConfigParser()
config.read("config.ini")
exif_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=config["redis"]["exif"],
)
xxhash_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=config["redis"]["xxhash"],
)


In [20]:
len(xxhash_cache.keys())


0

In [21]:
for key in xxhash_cache.keys():
    break


In [22]:
xxhash_cache.get(key)


NameError: name 'key' is not defined

In [23]:
def upsert(cls, get, set=None):    """    Interacting with Pony entities.    :param cls: The actual entity class    :param get: Identify the object (e.g. row) with this dictionary    :param set:     :return:    """    # does the object exist    assert isinstance(cls, EntityMeta), "{cls} is not a database entity".format(        cls=cls    )    # if no set dictionary has been specified    set = set or {}    if not cls.exists(**get):        # make new object        return cls(**set, **get)    else:        # get the existing object        obj = cls.get(**get)        for key, value in set.items():            obj.__setattr__(key, value)        return obj

In [24]:
set_sql_debug(False)
M = list()
for i, key in enumerate(exif_cache.keys()):
    if i % 10 == 0:
        print(i)
        db.commit()
    EXIF = exif_cache.get(key).decode()
    EXIF_ = json.loads(EXIF)
    g = {"xxhash": key.decode()}
    s = {"exif": EXIF_}
    M.append(upsert(Media, g, s))


In [25]:
M = select(c for c in Media if c.exif["EXIF:ISO"] == "250")


In [26]:
for m in M:
    break


In [27]:
m


NameError: name 'm' is not defined